In [64]:
import os 
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
print("Current Working Directory:", os.getcwd())
#the code is tested on vscode interactive window


#wandb.init(project="aml-final")

data = np.load('../Datasets/pathmnist.npz')

Current Working Directory: /Users/zarius/Documents/AMLS_assignment23_24/B


In [65]:
list(data.keys())

['train_images',
 'val_images',
 'test_images',
 'train_labels',
 'val_labels',
 'test_labels']

In [71]:
type(data['train_images'])
np.min(data['train_images']), np.max(data['train_images'])
data['train_images'].dtype

dtype('uint8')

In [73]:
train_images = data['train_images']
train_labels = data['train_labels']
val_images = data['val_images']
val_labels = data['val_labels']
test_images = data['test_images']
test_labels = data['test_labels']

In [74]:
from torch.utils.data import Dataset
import torch
from torchvision import transforms

from torchvision.transforms.functional import to_tensor

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Convert to tensor and scale to [0, 1]
        image = to_tensor(image)

        if self.transform:
            image = self.transform(image)

        return image, label


In [75]:
# Define transformations
transform = transforms.Compose([
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])  # Adjust these values based on your dataset

# Create datasets
train_dataset = CustomDataset(train_images, train_labels, transform=transform)
val_dataset = CustomDataset(val_images, val_labels, transform=transform)
test_dataset = CustomDataset(test_images, test_labels, transform=transform)


# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=32, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [76]:
data = next(iter(train_loader))
data[0].mean()

tensor(0.3406)

In [77]:
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np

for data, target in train_loader:
    #data = data.reshape(32,28,28,3)
    print(data.shape)
    #data = data.numpy()
    #imgplot = plt.imshow(data[0])
#    break

torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 28, 28])
torch.Size([32, 3, 2

In [78]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 9)  # 9 classes

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = torch.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = torch.softmax(x, dim=1)
  
        return output

In [83]:
model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 1
# Training loop
for epoch in range(num_epochs):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        target = target.reshape(-1)
        #predictions = torch.argmax(output, dim=1).reshape(-1,1)
        #print(predictions.shape)
        #print(target.shape)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss.item():.4f}')
    break
    # Validation loop
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data.reshape(32,3,28,28))
            target = target.reshape(-1).long()
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    print(f'Epoch: {epoch}, Validation Loss: {val_loss:.4f}, Accuracy: {100. * correct / len(val_loader.dataset):.2f}%')

Epoch: 0, Loss: 1.9866
